In [1]:
# Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# === GEO → CSV final para CARTO (lat/lon + the_geom) ===
!pip -q install geopy pandas tqdm

import os, unicodedata
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# -------- 0) Leer CSV base con perfiles ya calculados --------
READ_PATH = "/content/drive/MyDrive/Dataset_TFM/carto/adoptantes_perfil_geo.csv"  # <- ajusta si hace falta
df = pd.read_csv(READ_PATH)

# Copia de trabajo para GEO
df_geo = df.copy()

# Asegurar columnas lat/lon
for c in ["lat", "lon"]:
    if c not in df_geo.columns:
        df_geo[c] = pd.NA

# Dirección de búsqueda (municipio, provincia, España)
df_geo["poblacion"] = df_geo["poblacion"].astype(str).str.strip()
df_geo["provincia"] = df_geo["provincia"].astype(str).str.strip()
df_geo["direccion_busqueda"] = (
    df_geo["poblacion"] + ", " + df_geo["provincia"] + ", España"
).astype("string")

print("✅ Dataset cargado:", len(df_geo), "filas")

# -------- 1) Geocoder con caché (Nominatim + RateLimiter) --------
geolocator = Nominatim(user_agent="adoptapp_tfm")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, swallow_exceptions=True)

CACHE_PATH = "/content/drive/MyDrive/Dataset_TFM/adoptantes_cache_geocoding.csv"
if Path(CACHE_PATH).exists():
    cache = pd.read_csv(CACHE_PATH, dtype={"direccion_busqueda":"string"})
    print(f"Cache cargada ({len(cache)} filas).")
else:
    cache = pd.DataFrame(columns=["direccion_busqueda","lat","lon","display_name"])
    cache["direccion_busqueda"] = cache["direccion_busqueda"].astype("string")

# Únicas pendientes
unique_q = df_geo[["direccion_busqueda"]].drop_duplicates().astype({"direccion_busqueda":"string"})
pend = unique_q.merge(cache[["direccion_busqueda"]], on="direccion_busqueda", how="left", indicator=True)
pend = pend[pend["_merge"]=="left_only"].drop(columns="_merge").reset_index(drop=True)
print(f"Únicas totales: {len(unique_q)} | Pendientes: {len(pend)}")

batch, save_every = [], 50
for i, q in enumerate(tqdm(pend["direccion_busqueda"], desc="Geocodificando únicas"), start=1):
    try:
        loc = geocode(q, exactly_one=True, country_codes="es", addressdetails=False,
                      language="es", timeout=10)
        if loc:
            batch.append({"direccion_busqueda": q, "lat": loc.latitude, "lon": loc.longitude,
                          "display_name": getattr(loc, "address", None)})
        else:
            batch.append({"direccion_busqueda": q, "lat": pd.NA, "lon": pd.NA, "display_name": pd.NA})
    except Exception:
        batch.append({"direccion_busqueda": q, "lat": pd.NA, "lon": pd.NA, "display_name": pd.NA})

    if i % save_every == 0 or i == len(pend):
        tmp = pd.DataFrame(batch)
        cache = pd.concat([cache, tmp], ignore_index=True)
        cache.drop_duplicates(subset=["direccion_busqueda"], keep="last", inplace=True)
        cache.to_csv(CACHE_PATH, index=False)
        batch = []
print("Cache final guardada en:", CACHE_PATH)

# -------- 2) Merge coordenadas al dataset --------
df_geo = df_geo.merge(cache[["direccion_busqueda","lat","lon","display_name"]],
                      on="direccion_busqueda", how="left", suffixes=("", "_cache"))
for c in ["lat","lon","display_name"]:
    if f"{c}_cache" in df_geo.columns:
        df_geo[c] = df_geo[c].fillna(df_geo[f"{c}_cache"])
        df_geo.drop(columns=[f"{c}_cache"], inplace=True)

df_geo["lat"] = pd.to_numeric(df_geo["lat"], errors="coerce")
df_geo["lon"] = pd.to_numeric(df_geo["lon"], errors="coerce")

# (Opcional) fallback por provincia si quedan vacíos
# [...]

# -------- 3) the_geom en WKT --------
df_geo["the_geom"] = np.where(
    df_geo["lat"].notna() & df_geo["lon"].notna(),
    "POINT(" + df_geo["lon"].astype(str) + " " + df_geo["lat"].astype(str) + ")",
    None
)

# -------- 4) Normalización final (categorías limpias para CARTO) --------
def norm_title(s):
    if pd.isna(s): return s
    s = str(s).strip()
    s = unicodedata.normalize('NFKD', s).encode('ascii','ignore').decode('utf-8')
    return s[:1].upper() + s[1:].lower()

cols_cat_final = ["experiencia_animales","vivienda","horario_laboral","tiempo_disponible",
                  "genero","tipo_animal","poblacion","provincia","grupo_edad"]
for col in cols_cat_final:
    if col in df_geo.columns:
        df_geo[col] = df_geo[col].apply(norm_title)

# asegurar exactamente Baja/Media/Alta
if "experiencia_animales" in df_geo.columns:
    df_geo["experiencia_animales"] = (
        df_geo["experiencia_animales"].str.strip().str.lower()
        .map({"baja":"Baja","media":"Media","alta":"Alta"})
        .fillna(df_geo["experiencia_animales"])
    )

# -------- 5) Verificaciones previas a exportar --------
print(df_geo["perfil_adoptante"].value_counts(dropna=False).head())
assert df_geo["perfil_adoptante"].isna().sum() == 0, "Hay perfil_adoptante nulos"
print("Valores únicos en experiencia_animales:", df_geo["experiencia_animales"].unique())

# -------- 6) Exportar CSV final para CARTO --------
OUT_DIR = "/content/drive/MyDrive/Dataset_TFM/carto"
os.makedirs(OUT_DIR, exist_ok=True)

cols_out = [
    "id","poblacion","provincia","edad","genero","grupo_edad","vivienda","horario_laboral",
    "experiencia_animales","tiempo_disponible","tipo_animal","devuelto",
    "perfil_adoptante","apto",
    "lat","lon","the_geom","display_name"
]
cols_out = [c for c in cols_out if c in df_geo.columns]

OUT_PATH = os.path.join(OUT_DIR, "adoptantes_perfil_geo.csv")
df_geo[cols_out].to_csv(OUT_PATH, index=False, encoding="utf-8")
print("✅ CSV final para CARTO:", OUT_PATH, "| Filas:", len(df_geo))

✅ Dataset cargado: 2299 filas
Cache cargada (681 filas).
Únicas totales: 474 | Pendientes: 0


Geocodificando únicas: 0it [00:00, ?it/s]

Cache final guardada en: /content/drive/MyDrive/Dataset_TFM/adoptantes_cache_geocoding.csv
perfil_adoptante
Adulto experimentado    1610
Joven urbano             431
No apto                  258
Name: count, dtype: int64
Valores únicos en experiencia_animales: ['Baja' 'Media' 'Alta']
✅ CSV final para CARTO: /content/drive/MyDrive/Dataset_TFM/carto/adoptantes_perfil_geo.csv | Filas: 2299


/tmp/ipython-input-3336738764.py:77: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_geo[c] = df_geo[c].fillna(df_geo[f"{c}_cache"])
/tmp/ipython-input-3336738764.py:77: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_geo[c] = df_geo[c].fillna(df_geo[f"{c}_cache"])
